In [ ]:
pip install tensorflow tensorflow-hub tensorflow-datasets opencv-python-headless matplotlib


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Faster R-CNN modelini TensorFlow Hub'dan yükle
model_url = "https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1"
detector = hub.load(model_url)

# PASCAL VOC veri setini yükle (yalnızca küçük bir kısmını alıyoruz)
dataset, dataset_info = tfds.load('voc/2007', split='validation[:5%]', with_info=True)

# PASCAL VOC sınıf isimlerini al
category_index = {i: {'id': i, 'name': name} for i, name in enumerate(dataset_info.features['objects']['label'].names)}

# Bir örnek görüntü seç
sample = next(iter(dataset))
image_np = sample['image'].numpy()

# Görüntüyü hazırla
image_tensor = tf.convert_to_tensor(image_np)
image_tensor = tf.expand_dims(image_tensor, 0)

# 1. Region Proposal Generation
detections = detector(image_tensor)
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
detection_boxes = detections['detection_boxes']

# Region Proposals'ı görselleştir
def plot_region_proposals(image_np, boxes, max_boxes_to_draw=20):
    image_np_with_annotations = image_np.copy()
    for i in range(min(max_boxes_to_draw, boxes.shape[0])):
        box = tuple(boxes[i].tolist())
        image_np_with_annotations = cv2.rectangle(
            image_np_with_annotations,
            (int(box[1] * image_np.shape[1]), int(box[0] * image_np.shape[0])),
            (int(box[3] * image_np.shape[1]), int(box[2] * image_np.shape[0])),
            (0, 255, 0), 2
        )
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np_with_annotations)
    plt.title('Region Proposals')
    plt.show()

plot_region_proposals(image_np, detection_boxes)

#

# 3. Classification
detection_classes = detections['detection_classes'].astype(np.int64)
detection_scores = detections['detection_scores']

# Tespit sonuçlarını görselleştir
def plot_detections(image_np, boxes, classes, scores, category_index, max_boxes_to_draw=20, min_score_thresh=.5):
    image_np_with_annotations = image_np.copy()
    for i in range(min(max_boxes_to_draw, boxes.shape[0])):
        if scores[i] >= min_score_thresh:
            box = tuple(boxes[i].tolist())
            class_name = category_index[classes[i]]['name']
            display_str = f'{class_name}: {int(100 * scores[i])}%'
            image_np_with_annotations = cv2.rectangle(
                image_np_with_annotations,
                (int(box[1] * image_np.shape[1]), int(box[0] * image_np.shape[0])),
                (int(box[3] * image_np.shape[1]), int(box[2] * image_np.shape[0])),
                (0, 255, 0), 2
            )
            image_np_with_annotations = cv2.putText(
                image_np_with_annotations, display_str,
                (int(box[1] * image_np.shape[1]), int(box[0] * image_np.shape[0]) - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2
            )
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np_with_annotations)
    plt.title('Detected Objects')
    plt.show()

# Tespit edilen nesneleri görselleştir
plot_detections(image_np, detection_boxes, detection_classes, detection_scores, category_index)


In [ ]:
import os
import tarfile
import urllib.request

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

# PASCAL VOC veri setini indirmek için URL
VOC_URL = 'http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar'

# Veri setini indirme fonksiyonu
def download_and_extract_voc(url, download_dir):
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    tar_file_path = os.path.join(download_dir, 'VOCtrainval_06-Nov-2007.tar')
    if not os.path.exists(tar_file_path):
        print('Downloading PASCAL VOC 2007 dataset...')
        urllib.request.urlretrieve(url, tar_file_path)
        print('Download complete.')
    else:
        print('PASCAL VOC 2007 dataset already downloaded.')

    # Dosyayı çıkartma
    with tarfile.open(tar_file_path, 'r') as tar:
        tar.extractall(path=download_dir)
    print('Extraction complete.')

# Veri seti yolunu belirleyin
download_dir = './voc2007'
download_and_extract_voc(VOC_URL, download_dir)

# Görüntü ve anotasyon dosyalarını listeleme
voc_root = os.path.join(download_dir, 'VOCdevkit', 'VOC2007')
image_dir = os.path.join(voc_root, 'JPEGImages')
annotation_dir = os.path.join(voc_root, 'Annotations')

image_files = sorted([os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith('.jpg')])
annotation_files = sorted([os.path.join(annotation_dir, file) for file in os.listdir(annotation_dir) if file.endswith('.xml')])

# Bir örnek görüntü ve anotasyon dosyasını seçme
sample_image_path = image_files[0]
sample_annotation_path = annotation_files[0]

# Anotasyon dosyasını okuma
def parse_voc_annotation(annotation_path):
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    boxes = []
    classes = []
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        boxes.append([ymin, xmin, ymax, xmax])
        classes.append(class_name)
    return boxes, classes

# Örnek görüntü ve anotasyonu yükleme
image_np = cv2.imread(sample_image_path)
image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
boxes, classes = parse_voc_annotation(sample_annotation_path)

# Kategori indeksi oluşturma
unique_classes = sorted(set(classes))
category_index = {i: {'id': i, 'name': name} for i, name in enumerate(unique_classes)}

# Görüntüyü hazırla
image_tensor = tf.convert_to_tensor(image_np)
image_tensor = tf.expand_dims(image_tensor, 0)

# Faster R-CNN modelini TensorFlow Hub'dan yükle
model_url = "https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1"
detector = hub.load(model_url)

# 1. Region Proposal Generation
detections = detector(image_tensor)
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
detection_boxes = detections['detection_boxes']

# Region Proposals'ı görselleştir
def plot_region_proposals(image_np, boxes, max_boxes_to_draw=20):
    image_np_with_annotations = image_np.copy()
    for i in range(min(max_boxes_to_draw, boxes.shape[0])):
        box = tuple(boxes[i].tolist())
        image_np_with_annotations = cv2.rectangle(
            image_np_with_annotations,
            (int(box[1] * image_np.shape[1]), int(box[0] * image_np.shape[0])),
            (int(box[3] * image_np.shape[1]), int(box[2] * image_np.shape[0])),
            (0, 255, 0), 2
        )
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np_with_annotations)
    plt.title('Region Proposals')
    plt.show()

plot_region_proposals(image_np, detection_boxes)

# 2. Feature Extraction
# (Bu adım daha karmaşıktır ve genellikle derin öğrenme modelinin bir parçası olarak gerçekleştirilir)

# 3. Classification
detection_classes = detections['detection_classes'].astype(np.int64)
detection_scores = detections['detection_scores']

# Tespit sonuçlarını görselleştir
def plot_detections(image_np, boxes, classes, scores, category_index, max_boxes_to_draw=20, min_score_thresh=.5):
    image_np_with_annotations = image_np.copy()
    for i in range(min(max_boxes_to_draw, boxes.shape[0])):
        if scores[i] >= min_score_thresh:
            box = tuple(boxes[i].tolist())
            class_name = category_index[classes[i]]['name']
            display_str = f'{class_name}: {int(100 * scores[i])}%'
            image_np_with_annotations = cv2.rectangle(
                image_np_with_annotations,
                (int(box[1] * image_np.shape[1]), int(box[0] * image_np.shape[0])),
                (int(box[3] * image_np.shape[1]), int(box[2] * image_np.shape[0])),
                (0, 255, 0), 2
            )
            image_np_with_annotations = cv2.putText(
                image_np_with_annotations, display_str,
                (int(box[1] * image_np.shape[1]), int(box[0] * image_np.shape[0]) - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2
            )
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np_with_annotations)
    plt.title('Detected Objects')
    plt.show()

# Tespit edilen nesneleri görselleştir
plot_detections(image_np, detection_boxes, detection_classes, detection_scores, category_index)


In [ ]:
import os
import tarfile
import urllib.request

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

# PASCAL VOC veri setini indirmek için URL
VOC_URL = 'http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar'

# Veri setini indirme fonksiyonu
def download_and_extract_voc(url, download_dir):
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    tar_file_path = os.path.join(download_dir, 'VOCtrainval_06-Nov-2007.tar')
    if not os.path.exists(tar_file_path):
        print('Downloading PASCAL VOC 2007 dataset...')
        urllib.request.urlretrieve(url, tar_file_path)
        print('Download complete.')
    else:
        print('PASCAL VOC 2007 dataset already downloaded.')

    # Dosyayı çıkartma
    with tarfile.open(tar_file_path, 'r') as tar:
        tar.extractall(path=download_dir)
    print('Extraction complete.')

# Veri seti yolunu belirleyin
download_dir = './voc2007'
download_and_extract_voc(VOC_URL, download_dir)

# Görüntü ve anotasyon dosyalarını listeleme
voc_root = os.path.join(download_dir, 'VOCdevkit', 'VOC2007')
image_dir = os.path.join(voc_root, 'JPEGImages')
annotation_dir = os.path.join(voc_root, 'Annotations')

image_files = sorted([os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith('.jpg')])
annotation_files = sorted([os.path.join(annotation_dir, file) for file in os.listdir(annotation_dir) if file.endswith('.xml')])

# Bir örnek görüntü ve anotasyon dosyasını seçme
sample_image_path = image_files[0]
sample_annotation_path = annotation_files[0]

# Anotasyon dosyasını okuma
def parse_voc_annotation(annotation_path):
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    boxes = []
    classes = []
    for obj in root.findall('object'):
        class_name = obj.find('name').text
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        boxes.append([ymin, xmin, ymax, xmax])
        classes.append(class_name)
    return boxes, classes

# Örnek görüntü ve anotasyonu yükleme
image_np = cv2.imread(sample_image_path)
image_np = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
boxes, classes = parse_voc_annotation(sample_annotation_path)

# Kategori indeksi oluşturma
voc_classes = [
    "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair",
    "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant",
    "sheep", "sofa", "train", "tvmonitor"
]
category_index = {i+1: {'id': i+1, 'name': name} for i, name in enumerate(voc_classes)}

# Görüntüyü hazırla
image_tensor = tf.convert_to_tensor(image_np)
image_tensor = tf.expand_dims(image_tensor, 0)

# Faster R-CNN modelini TensorFlow Hub'dan yükle
model_url = "https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1"
detector = hub.load(model_url)

# 1. Region Proposal Generation
detections = detector(image_tensor)
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
detection_boxes = detections['detection_boxes']

# Region Proposals'ı görselleştir
def plot_region_proposals(image_np, boxes, max_boxes_to_draw=20):
    image_np_with_annotations = image_np.copy()
    for i in range(min(max_boxes_to_draw, boxes.shape[0])):
        box = tuple(boxes[i].tolist())
        image_np_with_annotations = cv2.rectangle(
            image_np_with_annotations,
            (int(box[1] * image_np.shape[1]), int(box[0] * image_np.shape[0])),
            (int(box[3] * image_np.shape[1]), int(box[2] * image_np.shape[0])),
            (0, 255, 0), 2
        )
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np_with_annotations)
    plt.title('Region Proposals')
    plt.show()

plot_region_proposals(image_np, detection_boxes)

# 2. Feature Extraction
# (Bu adım daha karmaşıktır ve genellikle derin öğrenme modelinin bir parçası olarak gerçekleştirilir)

# 3. Classification
detection_classes = detections['detection_classes'].astype(np.int64)
detection_scores = detections['detection_scores']

# Tespit sonuçlarını görselleştir
def plot_detections(image_np, boxes, classes, scores, category_index, max_boxes_to_draw=20, min_score_thresh=.5):
    image_np_with_annotations = image_np.copy()
    for i in range(min(max_boxes_to_draw, boxes.shape[0])):
        if scores[i] >= min_score_thresh:
            box = tuple(boxes[i].tolist())
            class_id = classes[i]
            if class_id in category_index:
                class_name = category_index[class_id]['name']
                display_str = f'{class_name}: {int(100 * scores[i])}%'
                image_np_with_annotations = cv2.rectangle(
                    image_np_with_annotations,
                    (int(box[1] * image_np.shape[1]), int(box[0] * image_np.shape[0])),
                    (int(box[3] * image_np.shape[1]), int(box[2] * image_np.shape[0])),
                    (0, 255, 0), 2
                )
                image_np_with_annotations = cv2.putText(
                    image_np_with_annotations, display_str,
                    (int(box[1] * image_np.shape[1]), int(box[0] * image_np.shape[0]) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2
                )
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np_with_annotations)
    plt.title('Detected Objects')
    plt.show()

# Tespit edilen nesneleri görselleştir
plot_detections(image_np, detection_boxes, detection_classes, detection_scores, category_index)
